<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مسئله‌ی دوم: پیش بینی هزینه‌ی خانوار در حوزه‌ی حمل و نقل
</font>
</h1>
</p>

In [205]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp
from scipy.stats import norm, expon, boxcox, shapiro
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [39]:
# file names and directories
data_dir = '../Data/'
R_files = ['R98', 'R99', 'R1400', 'R1401']
U_files = ['U98', 'U99', 'U1400', 'U1401']
all_files = R_files + U_files

### Prep data

In [174]:
# extract feautres
def extract_features(files_list):
    features = []
    for file_name in files_list:
        # features
        data = pd.read_excel(f'../Data/{file_name}.xlsx', sheet_name=(file_name+'Data'))
        data = pd.DataFrame(data)
        data = data[['Address', 'Fasl']]
        
        p1 = pd.read_excel(f'../Data/{file_name}.xlsx', sheet_name=(file_name+'P1'))
        p1 = p1.groupby("Address")['member'].count().reset_index()
        p1 = pd.DataFrame(p1)
        
        data = data.merge(p1,on='Address')
        
        p2 = pd.read_excel(f'../Data/{file_name}.xlsx', sheet_name=(file_name+'P2'))
        p2 = p2[['Address', 'vehicle', 'motorcycle', 'bicycle']]
        p2 = pd.DataFrame(p2)
        
        feature = data.merge(p2,on='Address')
        
        # target table
        p3s7 = pd.read_excel(f'../Data/{file_name}.xlsx', sheet_name=(file_name+'P3S07'))
        p3s7 = p3s7.groupby("Address")['value'].sum().reset_index()
        
        # TODO: fix this merge because loses some records with no values in p3s7 table
        feature = feature.merge(p3s7,on='Address')
                
        feature['Year'] = file_name[1:] 
        feature['Region'] = feature['Address'].apply(lambda x: str(x)[1:5])
        
        features.append(feature)
        
    return features

In [175]:
yearly_features = extract_features(all_files)

In [176]:
features = pd.concat([feature for feature in yearly_features])

In [177]:
features.describe(include='all')

,Address,Fasl,member,vehicle,motorcycle,bicycle,value,Year,Region
count,1.376000e+05,137600.000000,137600.000000,137600,137600,137600,1.376000e+05,137600,137600
unique,NaN,NaN,NaN,2,2,2,NaN,4,428
top,NaN,NaN,NaN,False,False,False,NaN,98,2301
freq,NaN,NaN,NaN,72413,104090,123779,NaN,35493,3821
mean,1.622540e+10,2.494150,3.524288,NaN,NaN,NaN,2.219055e+06,NaN,NaN
std,5.030487e+09,1.112771,1.390297,NaN,NaN,NaN,3.407234e+06,NaN,NaN
min,1.000100e+10,1.000000,1.000000,NaN,NaN,NaN,1.500000e+04,NaN,NaN
25%,1.150218e+10,1.000000,3.000000,NaN,NaN,NaN,6.500000e+05,NaN,NaN
50%,1.290136e+10,3.000000,4.000000,NaN,NaN,NaN,1.350000e+06,NaN,NaN
75%,2.130654e+10,3.000000,4.000000,NaN,NaN,NaN,2.700000e+06,NaN,NaN


In [178]:
features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 137600 entries, 0 to 18133
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Address     137600 non-null  int64 
 1   Fasl        137600 non-null  int64 
 2   member      137600 non-null  int64 
 3   vehicle     137600 non-null  bool  
 4   motorcycle  137600 non-null  bool  
 5   bicycle     137600 non-null  bool  
 6   value       137600 non-null  int64 
 7   Year        137600 non-null  object
 8   Region      137600 non-null  object
dtypes: bool(3), int64(4), object(2)
memory usage: 7.7+ MB


### Feature Engineering

In [182]:
# year column need to convert to labels
year_le = LabelEncoder()
features['Year'] = year_le.fit_transform(features['Year'])

In [191]:
year_classes = year_le.classes_

# Create a mapping from encoded labels to original classes
label_mapping = {index: label for index, label in enumerate(year_classes)}

# Print the mapping
print(label_mapping)

{0: '1400', 1: '1401', 2: '98', 3: '99'}


In [183]:
# province column need to convert to labels
region_le = LabelEncoder()
features['Region'] = region_le.fit_transform(features['Region'])


In [184]:
features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 137600 entries, 0 to 18133
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype
---  ------      --------------   -----
 0   Address     137600 non-null  int64
 1   Fasl        137600 non-null  int64
 2   member      137600 non-null  int64
 3   vehicle     137600 non-null  bool 
 4   motorcycle  137600 non-null  bool 
 5   bicycle     137600 non-null  bool 
 6   value       137600 non-null  int64
 7   Year        137600 non-null  int32
 8   Region      137600 non-null  int32
dtypes: bool(3), int32(2), int64(4)
memory usage: 6.7 MB


In [185]:
features.head()

,Address,Fasl,member,vehicle,motorcycle,bicycle,value,Year,Region
0,20001383908,2,4,False,True,False,750000,2,0
1,20001383911,2,2,False,False,False,370000,2,0
2,20001383915,2,2,True,False,False,1000000,2,0
3,20001383916,2,4,True,False,False,1050000,2,0
4,20001383920,2,5,False,False,False,350000,2,0


### Train Model

In [194]:
# split train test data
# Note: Winter 1401 is test data
test = features[(features['Year'] == 1) & (features['Fasl'] == 4)]
train = features[~((features['Year'] == 1) & (features['Fasl'] == 4))]
test.shape, train.shape

In [200]:
X_train = train.drop(['Address', 'value'], axis=1)
y_train = train['value']

X_test = test.drop(['Address', 'value'], axis=1)
y_test = test['value']

In [206]:
# new we train data
poly = PolynomialFeatures(degree=3)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

model = LinearRegression()
model.fit(X_train_poly, y_train)

LinearRegression()

### Test Model

In [ ]:
# TODO: check r_2 score and loss to validate model